In [1]:
import matplotlib
matplotlib.use('cairo')
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import jktools.plot as jplt
import jktools.moviepy as jmpy
import moviepy.editor as mpy

In [2]:
def plot_eye(
    center,
    fixation,
    eye_radius,
    fixation_radius,
    iris_percentage=0.7,
    pupil_percentage=0.4,
    ax=None,
    eye_kws=None,
    fix_kws=None,
    iris_kws=None,
    pupil_kws=None,
    tangent_kws=None):
    
    if not ax:
        ax = plt.gca()
        
    center = np.array(center) if not isinstance(center, np.ndarray) else center
    fixation = np.array(fixation) if not isinstance(fixation, np.ndarray) else fixation
    fixation_to_center = (fixation - center)
    fixation_to_center_distance = np.linalg.norm(fixation_to_center)
    direction_to_fixation = fixation_to_center / fixation_to_center_distance
    pupil_center = center + direction_to_fixation * eye_radius
    
    a = np.arcsin(fixation_radius / fixation_to_center_distance)
    b = np.arctan2(fixation_to_center[1], fixation_to_center[0])
    
    t_1 = b - a
    tangent_1 = np.array([ np.sin(t_1), -np.cos(t_1)]) * fixation_radius + fixation
    t_2 = b + a
    tangent_2 = np.array([-np.sin(t_2),  np.cos(t_2)]) * fixation_radius + fixation
    
    ax.plot([center[0], tangent_1[0]], [center[1], tangent_1[1]], **(tangent_kws if tangent_kws is not None else {}), zorder=3)
    ax.plot([center[0], tangent_2[0]], [center[1], tangent_2[1]], **(tangent_kws if tangent_kws is not None else {}), zorder=3)
    
    eye_patch = patches.Circle(
        center,
        eye_radius,
        zorder=4,
        **(dict((key, value) for key, value in eye_kws.items() if key!='edgecolor') if eye_kws is not None else {}),
        edgecolor=(0, 0, 0, 0)
    )
    ax.add_patch(eye_patch)
    
    eye_patch_upper = patches.Circle(
        center,
        eye_radius,
        zorder=10,
        **(dict((key, value) for key, value in eye_kws.items() if key!='facecolor') if eye_kws is not None else {}),
        facecolor=(0, 0, 0, 0)
    )
    ax.add_patch(eye_patch_upper)
    
    iris_patch = patches.Circle(
        pupil_center,
        eye_radius * iris_percentage,
        zorder=4,
        **(iris_kws if iris_kws is not None else {})
    )
    iris_patch.set_clip_path(eye_patch)
    ax.add_patch(iris_patch)
    
    pupil_patch = patches.Circle(
        pupil_center,
        eye_radius * pupil_percentage,
        zorder=4,
        **(pupil_kws if pupil_kws is not None else {})
    )
    pupil_patch.set_clip_path(eye_patch)
    ax.add_patch(pupil_patch)
    
#     reflex_patch = patches.Circle(
#         center + np.array([1, 1]) / np.sqrt(2) * eye_radius * 0.83,
#         eye_radius / 10,
#         zorder=9,
#         facecolor=(1, 1, 1, 0.8),
#         edgecolor=(0, 0, 0, 0)
#     )
#     ax.add_patch(reflex_patch)
    
    fixation_patch = patches.Circle(
        fixation,
        fixation_radius,
        zorder=4,
        **(fix_kws if fix_kws is not None else {})
    )
    ax.add_patch(fixation_patch)

In [24]:
a = jmpy.Animator()
a.add_animations(
    jmpy.Animation('x', np.array([0, 0, -5, -5, 6.5, 6.5, 4, 4, 0, 0]), np.diff(np.array([0, 1, 1.25, 2.25, 2.5, 2.65, 2.75, 4, 4.25, 4.75])), 'sin_io')
)

size = [2000, 1000]
def make_frame(t):
    plt.close('all')
    fig, ax = jplt.get_clean_fig_ax(size, xlims=[-10, 10], ylims=[-5, 5], facecolor=(1, 1, 1))
    a.update(t)
    
    led_xs = list(range(-9, 10))
    ax.scatter(led_xs, [3 for _ in led_xs], 5, color=(0.2, 0.2, 0.2))
    
    if 0.25 < t <= 1.25:
        ax.plot(-5, 3, 'og', markersize=6)
    if 1.25 < t <= 2.0:
        ax.plot(-5, 3, 'or', markersize=6)
    if 2.0 < t <= 2.4:
        ax.plot(7, 3, 'or', markersize=6)
    if 2.4 < t <= 3.0:
        ax.plot(4, 3, 'or', markersize=6)
    if 3.0 < t <= 3.9:
        ax.plot(4, 3, 'ob', markersize=6)
        
    black_tube = patches.Rectangle([-10, 1], 20, 4, facecolor='k', zorder=0)
    ax.add_patch(black_tube)
    
    plot_eye(
        [0, -3], [a['x'], 3], 1, 1.5,
        iris_percentage=0.7,
        pupil_percentage=0.4,
        eye_kws={'facecolor': 'w', 'edgecolor': 'k', 'linewidth':1},
        fix_kws={'facecolor': (0, 0, 0, 0), 'edgecolor': (0.7, 0.7, 0.7), 'linewidth':1, 'linestyle':'--'},
        iris_kws={'facecolor': (155/255, 102/255, 49/255), 'edgecolor': None, 'linewidth':1},
        pupil_kws={'facecolor': 'k', 'edgecolor': None, 'linewidth':1},
        tangent_kws={'color': (0.7, 0.7, 0.7), 'linewidth':1, 'linestyle':'--'},
        ax=ax
    )
    
    fig.canvas.draw()
    # return np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8).reshape([1000, 2000, 4])
    return jplt.rgba_buffer_cairo(fig, size)

clip = jmpy.RGBAVideoClip(make_frame, duration=7)
clip.write_videofile('fixation.mp4', fps=50, progress_bar=True, preset='veryslow')
#clip.ipython_display(fps=30, loop=True, autoplay=True)

[MoviePy] >>>> Building video fixation.mp4
[MoviePy] Writing video fixation.mp4




  0%|          | 0/351 [00:00<?, ?it/s]

  1%|          | 2/351 [00:00<00:47,  7.37it/s]

  1%|          | 3/351 [00:00<01:11,  4.90it/s]

  1%|          | 4/351 [00:00<01:12,  4.76it/s]

  1%|▏         | 5/351 [00:01<01:13,  4.72it/s]

  2%|▏         | 6/351 [00:01<01:13,  4.68it/s]

  2%|▏         | 7/351 [00:01<01:14,  4.62it/s]

  2%|▏         | 8/351 [00:01<01:14,  4.61it/s]

  3%|▎         | 9/351 [00:01<01:14,  4.58it/s]

  3%|▎         | 10/351 [00:02<01:14,  4.56it/s]

  3%|▎         | 11/351 [00:02<01:14,  4.56it/s]

  3%|▎         | 12/351 [00:02<01:14,  4.56it/s]

  4%|▎         | 13/351 [00:02<01:14,  4.56it/s]

  4%|▍         | 14/351 [00:03<01:14,  4.55it/s]

  4%|▍         | 15/351 [00:03<01:14,  4.54it/s]

  5%|▍         | 16/351 [00:03<01:13,  4.53it/s]

  5%|▍         | 17/351 [00:03<01:13,  4.53it/s]

  5%|▌         | 18/351 [00:03<01:13,  4.51it/s]

  5%|▌         | 19/351 [00:04<01:13,  4.51it/s]

  6%|▌         | 20/351 [00:04<01:13,  4.50it/s]

  6%|▌         

[MoviePy] Done.
[MoviePy] >>>> Video ready: fixation.mp4 



In [11]:
list(range(-9, 10))

[-9, -8, -7, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]